In [1]:
Name= 'Catherine Bui'
Organization= 'Center For Community Innovation'
Project= 'Twitter Displacement Study'

In [1]:
%%time
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import subprocess
from pathlib import Path
import csv
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import xlrd
import numpy as np
from io import StringIO
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric
from sqlalchemy.orm import sessionmaker
import dask.dataframe as das
import sqlite3
import datetime



CPU times: user 1.71 s, sys: 300 ms, total: 2.01 s
Wall time: 3.06 s


### READING THE CSV FILE 
1. Clean and prepare the data

In [19]:
%%time
file = "/scratch/public/catherinebui/sf_with_homeloc_one_tweet_per_day.csv"
df = das.read_csv(file)
df = df.rename(columns = {'sf_with_homeloc.csv': 'id'})

CPU times: user 76 ms, sys: 16 ms, total: 92 ms
Wall time: 87.7 ms


In [22]:
#Dropping Nan values
df=df.dropna(subset = ['home_tract'])
df = df.dropna(subset= ['tract'])
df['tract'] = df['tract'].astype('int')
df['home_tract'] = df['home_tract'].astype('int')

In [5]:
#Creating the neigbhor binary variable for each tweet 

#Creating a map of neigbhors to the source tract
sf_nn = pd.read_csv('BayArea_nearest_neighbor.csv')
def neighbors(src):
    for i in sf_nn['SRC_GEO_FI'].unique():
        if i == src:
            table=sf_nn[sf_nn['SRC_GEO_FI'] == i]
            return np.array(table['NBR_GEO_FI'])
    return []
n_ = []
for i in sf_nn['SRC_GEO_FI'].unique():
    n_.append(neighbors(i))
nn = pd.DataFrame(sf_nn['SRC_GEO_FI'].unique()) 
nn['neighbors'] = n_
nn = nn.rename(columns = {0: 'SRC_GEO_FI'})
neighbormap = dict(zip(nn['SRC_GEO_FI'], nn['neighbors']))

#Function that adds a 1 or 0 to each tweet if a tract is a neighbor or not of home_tract
def neighbor_check(x):
    if np.any(neighbormap.get(x['home_tract']) != None): 
        if x['tract'] in neighbormap.get(x['home_tract']):
            return 1
        else:
            return 0
    else:
        return 0
#Applying the neighbor_check to the dataframe
df['neighbor'] = df.apply(lambda x: neighbor_check(x), axis = 1, meta = 'float64')

#creating the non-neighbor table
other = df[(df['neighbor'] == 0) & (df['tract'] != df['home_tract'])]

In [6]:
demo_var = pd.read_csv('home_tract_variables.csv')

In [8]:
demo_var = demo_var.rename(
    columns = {'geo_fips': 'home_tract'})

In [9]:
#Joining the non-neighbor table with the home tract variables csv (demo_var)
other = other.merge(demo_var, on = 'home_tract', how = 'inner')

## Aggregation: 
Creating csv files for all the numbers and percentages of tweets sent from non-neighbor users in a tract with specific demographic characteristics. 

In [10]:
%%time
#aboverm_per_col15
aboverm_per_col15 = other.groupby(['tract', 'aboverm_per_col15']).aggregate(
    {'u_id': 'count'}).reset_index()
aboverm_per_col15 = aboverm_per_col15.categorize(columns = 'aboverm_per_col15')
aboverm_per_col15 = aboverm_per_col15.pivot_table(values = 'u_id',
                                                 columns = 'aboverm_per_col15', 
                                                 index = 'tract')

CPU times: user 50min 58s, sys: 4min 5s, total: 55min 3s
Wall time: 48min 48s


In [164]:
#Function to take the variable and return the csv of the counts of non-neighbor tweets with that condition
def create_csv(demo_variable):
    g = other.groupby(['tract', demo_variable]).aggregate(
        {'u_id': 'count'}).reset_index()
    g = g.categorize(columns = demo_variable)
    g = g.pivot_table(values = 'u_id',
                      columns = demo_variable, 
                      index = 'tract')
    g.to_csv(demo_variable + '_*.csv')

In [165]:
%%time
create_csv('aboverm_per_nonwhite15')


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

KeyboardInterrupt



In [14]:
%%time
create_csv('aboverm_empd14')

CPU times: user 1h 37min 19s, sys: 7min, total: 1h 44min 20s
Wall time: 1h 32min 41s


In [15]:
%%time
create_csv('aboverm_per_nhblk15')

CPU times: user 1h 35min 48s, sys: 6min 50s, total: 1h 42min 39s
Wall time: 1h 31min 9s


In [16]:
%%time
create_csv('aboverm_per_asian15')

CPU times: user 1h 34min 22s, sys: 6min 33s, total: 1h 40min 55s
Wall time: 1h 29min 45s


In [17]:
%%time
create_csv('aboverm_density15')

CPU times: user 1h 35min 39s, sys: 6min 54s, total: 1h 42min 33s
Wall time: 1h 31min 4s


In [18]:
%%time
create_csv('aboverm_per_hisp15')

CPU times: user 1h 34min 50s, sys: 6min 41s, total: 1h 41min 32s
Wall time: 1h 30min 15s


In [19]:
%%time
create_csv('LI_under80AMI')

CPU times: user 1h 34min 37s, sys: 6min 43s, total: 1h 41min 20s
Wall time: 1h 30min 8s


In [20]:
%%time
create_csv('HI_above120AMI')

CPU times: user 1h 34min 16s, sys: 6min 41s, total: 1h 40min 58s
Wall time: 1h 29min 43s


In [21]:
%%time
create_csv('MI_80_120AMI')

CPU times: user 1h 34min 20s, sys: 6min 36s, total: 1h 40min 57s
Wall time: 1h 29min 51s


In [22]:
%%time
create_csv('disp_type')

CPU times: user 1h 34min 28s, sys: 6min 41s, total: 1h 41min 9s
Wall time: 1h 29min 56s


In [23]:
%%time
aboverm_per_col15.to_csv('aboverm_per_col15_*.csv')

CPU times: user 46min 58s, sys: 3min 17s, total: 50min 16s
Wall time: 44min 47s


['aboverm_per_col15_0.csv']

In [29]:
[i + '_0.csv' for i in demo_var.columns if i != 'home_tract']

['aboverm_per_col15_0.csv',
 'aboverm_per_nonwhite15_0.csv',
 'aboverm_empd14_0.csv',
 'aboverm_per_nhblk15_0.csv',
 'aboverm_per_asian15_0.csv',
 'aboverm_density15_0.csv',
 'aboverm_per_hisp15_0.csv',
 'disp_type_0.csv',
 'LI_under80AMI_0.csv',
 'HI_above120AMI_0.csv',
 'MI_80_120AMI_0.csv']

## OUTPUT FILE
Creating the final result table. 

Labeling the columns with its correct name

Dividing the count by total tweets to get percentages

In [12]:
col15 = pd.read_csv('aboverm_per_col15_0.csv') # 0 and 1
nonwhite = pd.read_csv('aboverm_per_nonwhite15_0_1.csv') # 0 and 1
empd = pd.read_csv( 'aboverm_empd14_0.csv') # 0 and 1
nhblk = pd.read_csv( 'aboverm_per_nhblk15_0.csv') # 0 and 1
asian = pd.read_csv('aboverm_per_asian15_0.csv') # 0 and 1
density = pd.read_csv('aboverm_density15_0.csv') # 0 and 1
hisp = pd.read_csv('aboverm_per_hisp15_0.csv') # 0 and 1
disptype = pd.read_csv('disp_type_0.csv') # 9 variables
under80 = pd.read_csv('LI_under80AMI_0.csv') # 0 and 1
above120 = pd.read_csv('HI_above120AMI_0.csv') # 0 and 1
MI = pd.read_csv('MI_80_120AMI_0.csv') # 0 and 1

In [13]:
MI = MI[['tract', '1']]
MI = MI.rename({'1': 'ct_othertweets_MI_80_120AMI'}, axis =1)
above120 = above120[['tract', '1']].rename({'1': 'ct_othertweets_HI_above_120AMI'}, axis =1)
under80 = under80[['tract', '1']].rename({'1': 'ct_othertweets_LI_under80AMI'}, axis =1)
disptype.columns = ['tract'] + ['ct_othertweets_' + k for k in disptype.columns if k != 'tract']
hisp = hisp.rename({'1.0': 'ct_othertweets_aboverm_per_hisp15', 
                   '0.0': 'ct_othertweets_underm_per_hisp15'}, axis =1)
density = density.rename({'1': 'ct_othertweets_aboverm_density15', 
                   '0': 'ct_othertweets_underm_density15'}, axis =1)
asian = asian.rename({'1.0': 'ct_othertweets_aboverm_per_asian15',
                     '0.0': 'ct_othertweets_underm_per_asian15'}, axis =1)
nhblk = nhblk.rename({'1.0': 'ct_othertweets_aboverm_per_nhblk15',
                     '0.0': 'ct_othertweets_underm_per_nhblk15'}, axis = 1)
empd = empd.rename({'1': 'ct_othertweets_aboverm_empd14', 
                   '0': 'ct_othertweets_underm_empd14'}, axis = 1)
nonwhite = nonwhite.rename({'1.0': 'ct_othertweets_aboverm_per_nonwhite15',
                     '0.0': 'ct_othertweets_underm_per_nonwhite15'}, axis = 1)
col15 = col15.rename({'1.0': 'ct_othertweets_aboverm_per_col15',
                     '0.0': 'ct_othertweets_underm_per_col15'}, axis =1)

In [14]:
twitter_demo_sf= MI.merge(above120, on = 'tract', how = 'inner').merge(
under80, on = 'tract', how ='inner').merge(
disptype, on ='tract', how ='inner').merge(
hisp, on ='tract', how ='inner').merge(
density, on ='tract', how ='inner').merge(
asian, on ='tract', how = 'inner').merge(
nhblk, on = 'tract', how ='inner').merge(
empd, on ='tract', how ='inner').merge(
nonwhite, on ='tract', how = 'inner').merge(
col15, on ='tract', how = 'inner')

In [15]:

twitter_demo_sf = twitter_demo_sf.fillna(0)
original = pd.read_excel('Twitter_SF_outputfile_8_21.xlsx')
original = original[['tract', 'total tweets']]
twitter_demo_sf = twitter_demo_sf.merge(original, on = 'tract', how = 'inner')
for i in twitter_demo_sf.columns:
    if i not in ['total tweets', 'tract']:
        twitter_demo_sf['%_'+ i[2:len(i)]] = twitter_demo_sf[i]/twitter_demo_sf['total tweets']

In [21]:
twitter_demo_sf.head(10)

,tract,ct_othertweets_MI_80_120AMI,ct_othertweets_HI_above_120AMI,ct_othertweets_LI_under80AMI,ct_othertweets_Advanced Gentrification,ct_othertweets_College Town,ct_othertweets_LI - At Risk of Gentrification and/or Displacement,ct_othertweets_LI - Not Losing Low Income Households or Very Early Stages of Gentrification/Displacement,ct_othertweets_LI - Ongoing Gentrification,ct_othertweets_LI - Undergoing Displacement,...,%__othertweets_aboverm_per_asian15,%__othertweets_underm_per_nhblk15,%__othertweets_aboverm_per_nhblk15,%__othertweets_underm_empd14,%__othertweets_aboverm_empd14,%__othertweets_underm_per_nonwhite15,%__othertweets_aboverm_per_nonwhite15,%__othertweets_underm_per_col15,%__othertweets_aboverm_per_col15,%_tal tweets
0,6001400100,933.0,950.0,1061.0,128.0,131.0,227.0,553.0,600.0,89.0,...,0.173234,0.171450,0.261859,0.159554,0.278216,0.237026,0.196283,0.175167,0.258141,1.0
1,6001400200,1082.0,1136.0,1441.0,381.0,172.0,329.0,537.0,843.0,127.0,...,0.215787,0.182151,0.311406,0.106198,0.390072,0.262851,0.230707,0.161942,0.331615,1.0
2,6001400300,2399.0,2460.0,2566.0,346.0,240.0,787.0,1188.0,1779.0,143.0,...,0.139783,0.121585,0.224644,0.098241,0.249111,0.193628,0.152601,0.117094,0.229135,1.0
3,6001400400,878.0,587.0,850.0,204.0,153.0,177.0,437.0,552.0,37.0,...,0.118941,0.126544,0.187101,0.071419,0.242906,0.179090,0.134555,0.094501,0.219145,1.0
4,6001400500,395.0,431.0,744.0,102.0,164.0,178.0,247.0,383.0,33.0,...,0.103231,0.050141,0.159271,0.043572,0.166912,0.087009,0.122402,0.074139,0.135273,1.0
5,6001400600,517.0,320.0,341.0,36.0,38.0,77.0,177.0,428.0,14.0,...,0.038997,0.019241,0.081859,0.027830,0.073355,0.053513,0.047586,0.033671,0.067428,1.0
6,6001400700,975.0,737.0,1697.0,187.0,170.0,511.0,1004.0,710.0,74.0,...,0.073593,0.032320,0.120059,0.036886,0.115717,0.067998,0.084382,0.069923,0.082457,1.0
7,6001400800,648.0,383.0,819.0,101.0,62.0,260.0,439.0,522.0,31.0,...,0.077858,0.043229,0.167412,0.044720,0.167412,0.090127,0.120514,0.082330,0.128311,1.0
8,6001400900,250.0,268.0,384.0,33.0,22.0,124.0,131.0,224.0,21.0,...,0.066141,0.039282,0.111969,0.053550,0.097868,0.056572,0.094679,0.072855,0.078395,1.0
9,6001401000,1606.0,1307.0,2756.0,345.0,208.0,1075.0,1147.0,1291.0,165.0,...,0.084522,0.039150,0.144478,0.051789,0.131935,0.069484,0.114143,0.092689,0.090939,1.0


In [18]:
twitter_demo_sf.to_csv('Twitter_BayArea_w_demo_var_10_21.csv')